In [117]:
# from downloaded pages, extract features
# create a new db - extracted features. 
# columns -- Id (primary key), 
#            rewards -- save as a list 
#            n rewards
#            min rewards? 
#            n creator backed
# next -- has video, risks, faq -- did these features change? 
#                              focus on about this project to "risks"
#         body length, nlp -- typos
#
# Contains "Ships anywhere in the world"
# Video -- "Play", whitespace, and 00:00? --- not sure why I don't see some videos in older campaigns.
# limited rewards?

In [130]:
import requests
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode
from extract_kickstarter_features import *

In [133]:
%load_ext autoreload
%autoreload 2

In [3]:
con = mdb.connect('localhost','root','hobbes','kickstarter');

In [4]:
# how many rows do I have?
with con:
    cur = con.cursor()
    cur.execute("SELECT COUNT(Id) FROM Scraped_pages")
    rows = cur.fetchall()
    
    print rows[0]
    cur.close()

(99544,)


In [14]:
with con:
    cur = con.cursor()
    cur.execute("DESCRIBE Project_info")
    rows = cur.fetchall()
    for row in rows:
        print row[0]
    cur.close()

Id
Creator_name
Creator_url
Currency
Project_id
Category_name
Category_slug
Goal
Pledged
Project_date
Country
Location_club
Location_name
Blurb
N_backers
Name
URL
Launch_time
End_time


In [17]:
with con:
    cur = con.cursor()

    cur.execute("SELECT * from Project_info WHERE Category_name LIKE 'food%' LIMIT 10")
    rows = cur.fetchall()
    cur.close()

In [31]:
#for row in rows:
#    print row

In [24]:
# now look at an older campaign
with con:
    cur = con.cursor()

    cur.execute("SELECT * from Project_info WHERE Id = 95541 ")
    rows = cur.fetchall()
    cur.close()

In [25]:
for row in rows:
    print row

(95541, 'Jessica Welsch', 'https://www.kickstarter.com/profile/1715353522', 'USD', 104344008, 'Food', 'food', 25000, 25785, 'successful', 'US', 'minnetonka-mn', 'Minnetonka, MN', 'Make nutritious frozen whole food pops at home! The BellyFULL Kit gets kiddos interested in (and eating!) their fruits and veggies!', 150, 'The BellyFULL Kit ~ eat happily ever after!', 'https://www.kickstarter.com/projects/1715353522/the-bellyfull-kit-eat-happily-ever-after?ref=category', 1324053481, 1327112400)


In [30]:
# now look at an older, failed campaign
with con:
    cur = con.cursor()

    cur.execute("SELECT * from Project_info WHERE Id = 100541 ")
    rows = cur.fetchall()
    cur.close()

for row in rows:
    print row

(100541, 'Lindsey Mandel', 'https://www.kickstarter.com/profile/111998312', 'USD', 1518356875, 'Food', 'food', 1000, 415, 'failed', 'US', 'boulder-co', 'Boulder, CO', 'RollinGreens a mobile food service, is dedicated to providing quality, local, organic food to your neighborhood.', 8, 'RollinGreens, a Local Mobile Food Truck', 'https://www.kickstarter.com/projects/111998312/rollingreens-a-local-mobile-food-truck?ref=category', 1301083984, 1302024600)


In [ ]:
# test cases -- 
# 93541 (recent, funded), 
# 95541 (December 2011, funded), 
# 97541 (Dec 2014, failed)
# 100541 (April 2011, failed)

In [40]:
with con:
    cur = con.cursor()

    cur.execute("SELECT Creator_html from Scraped_pages WHERE Id = 100541")
    retrieved_text = cur.fetchall()
    cur.close()

In [42]:
creator_text = BeautifulSoup(retrieved_text[0][0])
creator_text_only = creator_text.get_text()
#print creator_text_only

In [44]:
get_creator_activity(creator_text_only)

0

In [98]:
with con:
    cur = con.cursor()

    cur.execute("SELECT Project_html from Scraped_pages WHERE Id = 93541")
    retrieved_text = cur.fetchall()
    cur.close()

In [99]:
this_soup = BeautifulSoup(retrieved_text[0][0])
project_text = this_soup.get_text()

In [47]:
print project_text








About this project



Ryan '"Ko" Cunningham and Lindsey Mandel are taking Cooking With Ko to the next level and need your help. We are launching RollinGreens, a mobile food truck specializing in global cuisine and bringing the focus on quality, local and organic ingredients from the farm to your table. We will provide foods for those with dietary restrictions, food allergies and those who just want to eat better for themselves and their family. We are making our menu items affordable and will offer a variety. 
We have already purchased our truck, and are currently designing and building our commercial mobile kitchen. We have a truck to pull the trailer. But need your help for things like Vinyl on the truck, kitchen supplies, cleaning supplies and a water tank.





FAQ






Have a question?
If the info above doesn't help, you can ask the project creator directly.

Ask a question





Report this project to Kickstarter






Rewards




You selected


$5 reward





0 backers




In [92]:
def extract_reward_level(project_text):
    # extract reward levels!
    matches = re.findall('\$(\d+) reward',project_text)
    #for match in matches:
    #    print match
    
    reward_int_list = [int(match) for match in matches]
    n_rewards = len(reward_int_list)
    
    return n_rewards, reward_int_list

In [138]:
rewards_predictors = extract_reward_level(project_text)

In [100]:
# go through a few kickstarters and see the number of rewards
with con:
    cur = con.cursor()
    
    for id in range(93541,93560):
        cur.execute("SELECT Project_html from Scraped_pages WHERE Id = %s",id)
        retrieved_text = cur.fetchall()
        
        this_soup = BeautifulSoup(retrieved_text[0][0])
        project_text = this_soup.get_text()
        print extract_reward_level(project_text)[0]
        
    cur.close()

13
4
11
0
10
9
9
0
0
6
6
14
8
8
7
7
5
5
19


In [139]:
a = map(list, rewards_predictors)

TypeError: 'int' object is not iterable

In [114]:
with con:
    cur = con.cursor()

    cur.execute("SELECT Project_html from Scraped_pages WHERE Id = 100541")
    retrieved_text = cur.fetchall()
    cur.close()

this_soup = BeautifulSoup(retrieved_text[0][0])
project_text = this_soup.get_text()

In [116]:
print this_soup

<div class="NS_projects__description_section">
<div class="py6 bg-white">
<div class="container-flex px2">
<div class="row">
<div class="col col-8">
<div class="mb6">
<h3 class="normal mb4 mobile-hide">
About this project
</h3>
</div>
<div class="full-description js-full-description responsive-media formatted-lists">
<p>Ryan '"Ko" Cunningham and Lindsey Mandel are taking Cooking With Ko to the next level and need your help. We are launching RollinGreens, a mobile food truck specializing in global cuisine and bringing the focus on quality, local and organic ingredients from the farm to your table. We will provide foods for those with dietary restrictions, food allergies and those who just want to eat better for themselves and their family. We are making our menu items affordable and will offer a variety. </p>
<p>We have already purchased our truck, and are currently designing and building our commercial mobile kitchen. We have a truck to pull the trailer. But need your help for things l

In [111]:
matches = re.findall('Estimated delivery:\n(\w+ \d+)',project_text)
if matches:
    print matches

[u'Jan 2015', u'Jan 2015', u'May 2015', u'Mar 2015', u'Jun 2015', u'Mar 2015', u'Mar 2015', u'Jul 2015', u'Jun 2015', u'Aug 2015', u'Jun 2015', u'Jul 2015', u'May 2015', u'Mar 2015', u'Jan 2015', u'Jun 2015', u'Jun 2015', u'Jun 2015', u'Apr 2015', u'May 2015']


In [ ]:
# how do I convert these to dates? time relative to launch?
# lower priority

In [118]:
print matches[0]

Jan 2015


In [135]:
print get_reward_levels(project_text)

(7, [5, 10, 25, 50, 100, 250, 500])


In [137]:
len(str(matches))

260